In [1]:
import sys
import os 
print(os.getcwd())
os.environ['CUDA_VISIBLE_DEVICES'] = "0"
current_dir = os.getcwd() 
print(f"カレントディレクトリ: {current_dir}")
project_root_path = os.path.dirname(current_dir)
print(f"プロジェクトルート: {project_root_path}")
if project_root_path not in sys.path:
    sys.path.append(project_root_path)
    
import torch
print(torch.cuda.is_available())



/workspace/project/analysis
カレントディレクトリ: /workspace/project/analysis
プロジェクトルート: /workspace/project
True


In [2]:
from project.models.make_model import single, early_fusion, slow_fusion, late_fusion
from project.dataloader.data_loader import WalkDataModule
from project.dataloader.data_loader_multi import MultiData
import yaml

from pytorch_lightning import Trainer

from IPython.display import clear_output

clear_output()

import torchmetrics
import torchmetrics

In [3]:
from pytorch_lightning import seed_everything

seed_everything(42, workers=True)

Seed set to 42


42

In [41]:
model_root = '/workspace/logs/resnet/single/2025-10-06/04-29-41'
model_path = '/workspace/logs/resnet/single/2025-10-06/04-29-41/fold3/version_0/checkpoints/15-2.37-0.5889.ckpt'
config_path = os.path.join(model_root, '.hydra/config.yaml')
AP_DATA_ROOT ='/workspace/data/out_dir_ver2_classified_side'
LAT_DATA_ROOT ='/workspace/data/out_dir_ver2_classified_side'
TARGET_FOLD = 'fold3'
INFERENCE_BATCH_SIZE = 4
SAVE_CAM_VALUES = True          
CAM_OUTPUT_DIR = "/workspace/data/analysis_logs/fails_points/20251023/single_side/CAM"  


In [42]:
def load_config(config_path):
    """YAMLファイルを読み込む関数"""
    try:
        with open(config_path, 'r') as f:
            config = yaml.safe_load(f)
        print(f"設定ファイル '{config_path}' を正常に読み込みました。")
        return config
    except FileNotFoundError:
        print(f"❌ エラー: 設定ファイル '{config_path}' が見つかりません。")
        sys.exit()

class HParams:
    """config.yamlの階層構造を模倣するクラス"""
    def __init__(self, data_config, model_config, train_config):
        self.data = type('DataConfig', (object,), data_config)()
        self.model = type('ModelConfig', (object,), model_config)()
        self.train = type('TrainConfig', (object,), train_config)()
config_data = load_config(config_path)
try:
    hparams = HParams(
        config_data['data'], 
        config_data['model'], 
        config_data['train']
    )
    print("✅ HParamsオブジェクトを正常に作成しました。")
except KeyError as e:
    print(f"❌ エラー: config.yaml に必要なキー {e} が見つかりません。")
    sys.exit()

設定ファイル '/workspace/logs/resnet/single/2025-10-06/04-29-41/.hydra/config.yaml' を正常に読み込みました。
✅ HParamsオブジェクトを正常に作成しました。


In [43]:
# ==============================================================================
# ## 3. モデルの準備：インスタンス化と学習済み重みのロード
# ==============================================================================
print("\nステップ3: モデルを準備します...")

# --- 設定値の取得 ---
fusion_type = hparams.train.experiment
device_ids = 0

# --- デバイスの設定 ---
if torch.cuda.is_available():
    # 指定されたGPU IDが利用可能かチェック
    if device_ids >= torch.cuda.device_count():
        print(f"\n❌ エラー: 指定されたGPU ID {device_ids} は利用できません。利用可能なGPUは {torch.cuda.device_count()}台です。")
        sys.exit()
    
    device = torch.device(f"cuda:{device_ids}")
    print(f"GPU {device_ids} を使用します。")
else:
    device = torch.device("cpu")
    print("GPUが利用できないため、CPUを使用します。")

# --- モデルのインスタンス化 ---
if fusion_type == 'single':
    model = single(hparams).to(device)
elif fusion_type == 'early_fusion':
    model = early_fusion(hparams).to(device)
elif fusion_type == 'late_fusion':
    model = late_fusion(hparams).to(device)
elif fusion_type == 'slow_fusion':
    model = slow_fusion(hparams).to(device)
else:
    raise ValueError(f"不明なフュージョンタイプ: {fusion_type}")
print(f"'{fusion_type}' モデルをインスタンス化し、デバイス '{device}' に配置しました。")



ステップ3: モデルを準備します...
GPU 0 を使用します。
'single' モデルをインスタンス化し、デバイス 'cuda:0' に配置しました。


Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0
/opt/conda/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [44]:
# --- 学習済み重みのロード（最終修正版） ---
try:
    checkpoint = torch.load(model_path, map_location=device, weights_only=False)
    
    if 'state_dict' in checkpoint:
        original_state_dict = checkpoint['state_dict']
        new_state_dict = {}
        # キーの先頭にある "model." プレフィックスを削除
        for k, v in original_state_dict.items():
            if k.startswith("model."):
                new_key = k[len("model."):]
            else:
                new_key = k
            new_state_dict[new_key] = v
        
        model.load_state_dict(new_state_dict)
    else:
        model.load_state_dict(checkpoint)
        
    model.eval() # モデルを評価モードに設定
    print("✅ 学習済み重みをモデルに正常にロードしました。")

except Exception as e:
    import traceback
    print(f"\n❌ エラー: モデルの重みロード中にエラーが発生しました: {e}")
    print(traceback.format_exc())
    sys.exit()

✅ 学習済み重みをモデルに正常にロードしました。


In [45]:
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.image import show_cam_on_image
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget

results = {}

# ### ケース1: 2つの動画を入力するフュージョンモデルの場合 ###
if fusion_type in ['late_fusion', 'early_fusion', 'slow_fusion']:
    print(f"'{fusion_type}' モデル（2入力）の推論パイプラインを開始します。")
    hparams.data.ap_data_path, hparams.data.lat_data_path = AP_DATA_ROOT, LAT_DATA_ROOT
    hparams.train.current_fold = TARGET_FOLD
    hparams.data.batch_size, hparams.data.num_workers = INFERENCE_BATCH_SIZE, 0
    
    datamodule = MultiData(hparams)
    print(f"'{TARGET_FOLD}' の検証(val)データを読み込んでいます...")
    datamodule.setup(stage='test') 
    test_loader = datamodule.test_dataloader()
    print("データローダーの準備が完了しました。")

    with torch.no_grad():
        for i, batch in enumerate(test_loader):
            print(f"\n--- [バッチ {i+1}] 推論中 ---")
            batch_dict = batch[0]
            inputs_front, inputs_side = batch_dict["ap"]["video"].to(device), batch_dict["lat"]["video"].to(device)
            video_names = batch_dict["ap"]["video_name"]
            if batch_dict['ap']['video_name'] != batch_dict['lat']['video_name']:
                print('error')
            outputs = model(inputs_front, inputs_side)
            prob_for_class_1 = torch.sigmoid(outputs)
            predicted_indices = (prob_for_class_1 > 0.5).long()
            
            for idx in range(len(video_names)):
                video_name = os.path.basename(video_names[idx])
                pred_class, prob1 = predicted_indices[idx].item(), prob_for_class_1[idx].item()
                confidence = prob1 if pred_class == 1 else 1 - prob1
                results[video_name] = {'prediction': pred_class, 'confidence': f"{confidence:.4f}", 'prob_for_class_1': f"{prob1:.4f}"}
                print(f"  - {video_name}: Logit: {outputs[idx].item():.4f} -> P(class 1)={prob1:.4f} -> 予測クラス: {pred_class} (確信度: {confidence*100:.2f}%)")

# 'single' モデル（1入力）の推論パイプライン
elif fusion_type == 'single':
    
    # --- CAM保存設定 ---
    # ▼▼▼【設定項目】保存するCAMの対象を選択 ▼▼▼
    # 'all'                  -> 全てのビデオのCAMを保存
    # 'class_0_predictions'  -> モデルが「0」と予測したビデオのCAMのみ保存
    # 'class_1_predictions'  -> モデルが「1」と予測したビデオのCAMのみ保存
    SAVE_CAM_FOR = 'all'

    print(f"'single' モデル（1入力）の推論パイプラインを開始します。")
    # --- データ準備 (変更なし) ---
    hparams.data.ap_data_path, hparams.data.lat_data_path = AP_DATA_ROOT, LAT_DATA_ROOT
    hparams.train.current_fold = TARGET_FOLD
    hparams.data.batch_size, hparams.data.num_workers = INFERENCE_BATCH_SIZE, 0
    datamodule = WalkDataModule(hparams)
    print(f"'{TARGET_FOLD}' の検証(val)データを読み込んでいます...")
    datamodule.setup(stage='test')
    test_loader = datamodule.test_dataloader()
    print("データローダーの準備が完了しました。")

    # --- 2. Grad-CAMの準備 (変更なし) ---
    try:
        cam_model = model.resnet_model
        target_layers = [cam_model.layer4[-1]]
        cam = GradCAM(model=cam_model, target_layers=target_layers)
        print(f"Grad-CAMの準備が完了しました。")
    except Exception as e:
        print(f"エラー: Grad-CAMのターゲット層が見つかりません。詳細: {e}")
        cam = None

    # --- 3. 推論ループとGrad-CAMの実行 ---
    if test_loader is not None:
        saved_count = 0
        with torch.no_grad():
            for i, batch in enumerate(test_loader):
                print(f"\n--- [バッチ {i+1}] 推論中 ---")
                video_5d_original_shape = batch["video"].to(device)
                video_names = batch["video_name"]
                input_video_for_model = video_5d_original_shape.permute(0, 2, 1, 3, 4).contiguous()
                
                model.eval()
                outputs = model(input_video_for_model)
                prob_for_class_1 = torch.sigmoid(outputs)
                predicted_indices = (prob_for_class_1 > 0.5).long()
                
                if cam is not None:
                    # バッチ内の各ビデオに対して処理
                    for vid_idx_in_batch in range(len(video_names)):
                        prediction = predicted_indices[vid_idx_in_batch].item()
                        
                        # ▼▼▼【ここから修正】設定に基づいて保存するかどうかを判断 ▼▼▼
                        should_save = False
                        if SAVE_CAM_FOR == 'all':
                            should_save = True
                        elif SAVE_CAM_FOR == 'class_0_predictions' and prediction == 0:
                            should_save = True
                        elif SAVE_CAM_FOR == 'class_1_predictions' and prediction == 1:
                            should_save = True

                        if should_save:
                            if saved_count % 10 == 0: # ログが多すぎないように調整
                                print(f"\n--- [バッチ {i+1}] のCAMを生成・保存中... ---")
                            
                            with torch.enable_grad():
                                targets = [ClassifierOutputTarget(0)] # 常にクラス1の根拠を計算
                                B, C, T, H, W = video_5d_original_shape.shape
                                
                                # このビデオ単体でCAMを計算 (バッチ処理を避ける)
                                single_video_input = video_5d_original_shape[vid_idx_in_batch].unsqueeze(0)
                                video_permuted = single_video_input.permute(0, 2, 1, 3, 4).contiguous()
                                input_tensor_4d = video_permuted.view(1 * T, C, H, W)
                                
                                grayscale_cam = cam(input_tensor=input_tensor_4d, targets=targets)
                            
                            # --- 非ゼロCAMを平均化 ---
                            non_zero_cams = [frame for frame in grayscale_cam if np.sum(frame) > 0]
                            if non_zero_cams:
                                averaged_cam = np.mean(non_zero_cams, axis=0)
                            else:
                                averaged_cam = np.zeros_like(grayscale_cam[0])

                            # --- 保存 ---
                            fold_name = TARGET_FOLD
                            fold_output_path = os.path.join(CAM_OUTPUT_DIR, fold_name)
                            os.makedirs(fold_output_path, exist_ok=True)
                            
                            video_name_short = os.path.basename(video_names[vid_idx_in_batch])
                            base_filename, _ = os.path.splitext(video_name_short)
                            save_filename = f"cam_average_{base_filename}_pred_{prediction}.npy"
                            full_save_path = os.path.join(fold_output_path, save_filename)
                            
                            np.save(full_save_path, averaged_cam)
                            saved_count += 1
                    # ▲▲▲【修正ここまで】▲▲▲

                # --- 推論結果の出力 (変更なし) ---
                for idx in range(len(video_names)):
                    video_name = os.path.basename(video_names[idx])
                    pred_class = predicted_indices[idx].item()
                    prob1 = prob_for_class_1[idx].item()
                    confidence = prob1 if pred_class == 1 else 1 - prob1
                    results[video_name] = {'prediction': pred_class, 'confidence': f"{confidence:.4f}", 'prob_for_class_1': f"{prob1:.4f}"}
                    print(f"  - {video_name}: Logit: {outputs[idx].item():.4f} -> P(class 1)={prob1:.4f} -> 予測クラス: {pred_class} (確信度: {confidence*100:.2f}%)")
            
            if cam is not None:
                print(f"\n--- 処理完了 ---")
                print(f"条件に一致した {saved_count} 個の平均CAMを保存しました。")

else:
    print(f"エラー: fusion_type '{fusion_type}' はこのスクリプトでサポートされていません。")

# --- 4.4. 結果の集計と表示 ---
print("\n\n==================== 全ての推論結果 ====================")
if not results:
    print("推論が実行された動画はありませんでした。")
else:
    results_df = pd.DataFrame.from_dict(results, orient='index')
    results_df.index.name = 'Video Filename'
    results_df = results_df.reset_index()
    display(results_df)

'single' モデル（1入力）の推論パイプラインを開始します。
'fold3' の検証(val)データを読み込んでいます...
データローダーの準備が完了しました。
Grad-CAMの準備が完了しました。

--- [バッチ 1] 推論中 ---

--- [バッチ 1] のCAMを生成・保存中... ---
  - ASD_Right_20200127_1728_5-7.mp4: Logit: -0.3827 -> P(class 1)=0.4055 -> 予測クラス: 0 (確信度: 59.45%)
  - ASD_Left_20200127_1685_4-10.mp4: Logit: 0.4775 -> P(class 1)=0.6172 -> 予測クラス: 1 (確信度: 61.72%)
  - ASD_Left_20190917_1995_1-4.mp4: Logit: -1.5250 -> P(class 1)=0.1787 -> 予測クラス: 0 (確信度: 82.13%)
  - ASD_Left_20180625_1_1427_2-6.mp4: Logit: 1.3374 -> P(class 1)=0.7921 -> 予測クラス: 1 (確信度: 79.21%)

--- [バッチ 2] 推論中 ---
  - HipOA_Right_20180806_2_883_3-4.mp4: Logit: -4.7851 -> P(class 1)=0.0083 -> 予測クラス: 0 (確信度: 99.17%)
  - DHS_Left_20200528_1119_4-8.mp4: Logit: -6.9129 -> P(class 1)=0.0010 -> 予測クラス: 0 (確信度: 99.90%)
  - DHS_Right_20160927_1098_3-3.mp4: Logit: -6.2353 -> P(class 1)=0.0020 -> 予測クラス: 0 (確信度: 99.80%)
  - LCS_Left_20211012_780_2-5.mp4: Logit: -5.9805 -> P(class 1)=0.0025 -> 予測クラス: 0 (確信度: 99.75%)

--- [バッチ 3] 推論中 ---

--- [バッチ 

,Video Filename,prediction,confidence,prob_for_class_1
0,ASD_Right_20200127_1728_5-7.mp4,0,0.5945,0.4055
1,ASD_Left_20200127_1685_4-10.mp4,1,0.6172,0.6172
2,ASD_Left_20190917_1995_1-4.mp4,0,0.8213,0.1787
3,ASD_Left_20180625_1_1427_2-6.mp4,1,0.7921,0.7921
4,HipOA_Right_20180806_2_883_3-4.mp4,0,0.9917,0.0083
...,...,...,...,...
1714,HipOA_Right_20180806_2_865_2-4.mp4,1,0.5914,0.5914
1715,DHS_Right_20160927_1114_3-3.mp4,0,0.9220,0.0780
1716,ASD_Left_20190917_2001_2-4.mp4,0,0.9035,0.0965
1717,ASD_Right_20180625_1_1420_4-4.mp4,0,0.9812,0.0188


In [47]:
# 【分析用コード：修正版】
import numpy as np
import pandas as pd
import os
import glob

# --- 【設定】分析したいFoldを指定してください ---
TARGET_FOLD_TO_ANALYZE = TARGET_FOLD
CAM_OUTPUT_DIR = "/workspace/data/analysis_logs/fails_points/20251023/single_side/CAM"

# --- 1. 分析対象の.npyファイルをすべて取得 ---
fold_path = os.path.join(CAM_OUTPUT_DIR, TARGET_FOLD_TO_ANALYZE)
npy_files = glob.glob(os.path.join(fold_path, 'cam_average_*.npy')) # ファイル名を 'cam_average_' でフィルタリング

if not npy_files:
    print(f"エラー: '{fold_path}' に 'cam_average_*.npy' ファイルが見つかりませんでした。")
else:
    print(f"'{fold_path}' から {len(npy_files)} 個の平均CAMファイルを検出しました。分析を開始します...\n")
    
    all_results = []

    # --- 2. 各.npyファイルをループして分析 ---
    for file_path in npy_files:
        video_filename = os.path.basename(file_path).replace('cam_average_', '').replace('.npy', '')
        
        # 平均化されたCAMデータ(2D配列)を読み込み
        averaged_cam_data = np.load(file_path)
        
        # ヒートマップを3つの領域に分割
        height = averaged_cam_data.shape[0]
        one_third, two_thirds = height // 3, 2 * (height // 3)
        top_region = averaged_cam_data[0:one_third, :]
        middle_region = averaged_cam_data[one_third:two_thirds, :]
        bottom_region = averaged_cam_data[two_thirds:, :]
        
        # 各領域の注目度スコアを計算
        top_score, middle_score, bottom_score = top_region.sum(), middle_region.sum(), bottom_region.sum()
        total_score = top_score + middle_score + bottom_score
        
        scores = {"上部": top_score, "中部": middle_score, "下部": bottom_score}
        dominant_region = max(scores, key=scores.get) if total_score > 0 else "N/A"
        
        # 割合を計算
        if total_score > 0:
            top_percent = (top_score / total_score) * 100
            middle_percent = (middle_score / total_score) * 100
            bottom_percent = (bottom_score / total_score) * 100
        else:
            top_percent, middle_percent, bottom_percent = 0, 0, 0
        
        # 結果を辞書として保存
        result_dict = {
            'video_name': video_filename,
            'top_percent': round(top_percent, 2),
            'middle_percent': round(middle_percent, 2),
            'bottom_percent': round(bottom_percent, 2),
            'dominant_region': dominant_region
        }
        all_results.append(result_dict)

    # --- 3. 結果をPandas DataFrameに変換して表示 ---
    results_df = pd.DataFrame(all_results)
    print("--- 平均CAMの注目領域 分析結果 ---")
    display(results_df)
    
    # --- 4. Fold全体のサマリーを計算して表示 ---
    print(f"\n--- Fold '{TARGET_FOLD_TO_ANALYZE}' 全体のサマリー ---")
    dominant_summary = results_df['dominant_region'].value_counts()
    print("\n[主な注目領域の出現回数]")
    print(dominant_summary)
    average_attention = results_df[['top_percent', 'middle_percent', 'bottom_percent']].mean()
    print("\n[各領域の平均注目率 (%)]")
    print(average_attention)

'/workspace/data/analysis_logs/fails_points/20251023/single_side/CAM/fold3' から 1719 個の平均CAMファイルを検出しました。分析を開始します...

--- 平均CAMの注目領域 分析結果 ---


,video_name,top_percent,middle_percent,bottom_percent,dominant_region
0,DHS_Right_20200528_1135_6-6_pred_0,98.529999,1.470000,0.00,上部
1,ASD_Left_20190917_1960_3-4_pred_0,47.779999,52.220001,0.00,中部
2,ASD_Right_20190819_1452_2-3_pred_0,33.009998,66.989998,0.00,中部
3,HipOA_Left_20180806_2_856_1-5_pred_0,31.120001,68.879997,0.00,中部
4,ASD_Right_20200127_1690_5-7_pred_0,51.810001,48.189999,0.00,上部
...,...,...,...,...,...
1714,HipOA_Right_20180806_2_887_2-4_pred_0,77.339996,22.660000,0.00,上部
1715,HipOA_Left_20180806_2_884_6-6_pred_0,30.420000,69.580002,0.00,中部
1716,ASD_Left_20190917_1972_1-4_pred_0,32.950001,66.980003,0.07,中部
1717,DHS_Right_20160927_1070_2-3_pred_0,87.779999,12.220000,0.00,上部



--- Fold 'fold3' 全体のサマリー ---

[主な注目領域の出現回数]
dominant_region
中部     930
上部     606
N/A    143
下部      40
Name: count, dtype: int64

[各領域の平均注目率 (%)]
top_percent       46.465625
middle_percent    42.347399
bottom_percent     2.868185
dtype: float64


In [ ]:
import numpy as np
import pandas as pd
import os
import glob

# --- 【設定】---
# CAMデータが保存されている親ディレクトリ

# --- 1. すべてのFoldディレクトリを自動的に検出 ---
# CAM_OUTPUT_DIR内のサブディレクトリをFold名として取得
CAM_OUTPUT_DIR = '/workspace/data/analysis_logs/fails_points/20251023/single_front/CAM'
# is_dir()でディレクトリのみを対象とする
try:
    fold_dirs = [d.name for d in os.scandir(CAM_OUTPUT_DIR) if d.is_dir()]
    if not fold_dirs:
        print(f"エラー: '{CAM_OUTPUT_DIR}'内にFoldのサブディレクトリが見つかりませんでした。")
        # Exit the script if no folds are found
        # In a notebook, you might just print the error and stop.
    else:
        print(f"分析対象のFoldを検出しました: {sorted(fold_dirs)}\n")
except FileNotFoundError:
    print(f"エラー: 親ディレクトリ'{CAM_OUTPUT_DIR}'が見つかりません。")
    fold_dirs = []


# --- 2. 各ファイルの分析結果を保存するリストを準備 ---
all_results = []

# --- 3. 検出した各Foldをループして分析 ---
if fold_dirs:
    for fold_name in sorted(fold_dirs):
        fold_path = os.path.join(CAM_OUTPUT_DIR, fold_name)
        npy_files = glob.glob(os.path.join(fold_path, 'cam_average_*.npy'))
        
        if not npy_files:
            print(f"--- Fold '{fold_name}' には分析対象の.npyファイルがありませんでした。スキップします。 ---")
            continue
            
        print(f"--- Fold '{fold_name}' の分析中 ({len(npy_files)}個のファイル)... ---")

        for file_path in npy_files:
            video_filename = os.path.basename(file_path).replace('cam_average_', '').replace('.npy', '')
            averaged_cam_data = np.load(file_path)
            
            # --- 注目領域の計算 (変更なし) ---
            height = averaged_cam_data.shape[0]
            one_third, two_thirds = height // 3, 2 * (height // 3)
            top_region = averaged_cam_data[0:one_third, :]
            middle_region = averaged_cam_data[one_third:two_thirds, :]
            bottom_region = averaged_cam_data[two_thirds:, :]
            
            top_score, middle_score, bottom_score = top_region.sum(), middle_region.sum(), bottom_region.sum()
            total_score = top_score + middle_score + bottom_score
            
            scores = {"上部": top_score, "中部": middle_score, "下部": bottom_score}
            dominant_region = max(scores, key=scores.get) if total_score > 0 else "N/A"
            
            if total_score > 0:
                top_percent = (top_score / total_score) * 100
                middle_percent = (middle_score / total_score) * 100
                bottom_percent = (bottom_score / total_score) * 100
            else:
                top_percent, middle_percent, bottom_percent = 0, 0, 0
            
            # 結果を辞書として保存 (Fold名も追加)
            result_dict = {
                'fold': fold_name,
                'video_name': video_filename,
                'top_percent': round(top_percent, 2),
                'middle_percent': round(middle_percent, 2),
                'bottom_percent': round(bottom_percent, 2),
                'dominant_region': dominant_region
            }
            all_results.append(result_dict)

    # --- 4. 全Foldの結果を一つのDataFrameにまとめて表示 ---
    if all_results:
        results_df = pd.DataFrame(all_results)
        
        print("\n\n=============================================")
        print("=== 全Foldの詳細な分析結果（最初の15件を表示） ===")
        print("=============================================")
        display(results_df.head(3000))

        # --- 5. Foldごとのサマリーを計算して表示 ---
        print("\n\n============================")
        print("=== Foldごとのサマリー ===")
        print("============================")
        for fold_name in sorted(fold_dirs):
            fold_df = results_df[results_df['fold'] == fold_name]
            if not fold_df.empty:
                print(f"\n--- Fold '{fold_name}' ---")
                dominant_summary = fold_df['dominant_region'].value_counts()
                print("[主な注目領域の出現回数]")
                print(dominant_summary)
                average_attention = fold_df[['top_percent', 'middle_percent', 'bottom_percent']].mean()
                print("\n[各領域の平均注目率 (%)]")
                print(average_attention)
        
        # --- 6. 全Foldを統合した総合サマリーを表示 ---
        print("\n\n======================================")
        print("=== 全Fold統合 総合サマリー ===")
        print("======================================")
        
        # (1) 全体の主な注目領域の出現回数
        total_dominant_summary = results_df['dominant_region'].value_counts()
        print("\n[全体の主な注目領域の出現回数]")
        print(total_dominant_summary)
        
        # (2) 全体の各領域の平均注目度
        total_average_attention = results_df[['top_percent', 'middle_percent', 'bottom_percent']].mean()
        print("\n[全体の各領域の平均注目率 (%)]")
        print(total_average_attention)
        
        # (オプション) 全結果をCSVファイルとして保存
        # results_df.to_csv("cam_analysis_all_folds.csv", index=False)
        # print("\n全分析結果を 'cam_analysis_all_folds.csv' に保存しました。")

分析対象のFoldを検出しました: ['fold0', 'fold1', 'fold2', 'fold3', 'fold4']

--- Fold 'fold0' の分析中 (2364個のファイル)... ---
--- Fold 'fold1' の分析中 (2896個のファイル)... ---
--- Fold 'fold2' の分析中 (2473個のファイル)... ---
--- Fold 'fold3' の分析中 (1719個のファイル)... ---
--- Fold 'fold4' の分析中 (1762個のファイル)... ---


=== 全Foldの詳細な分析結果（最初の15件を表示） ===


,fold,video_name,top_percent,middle_percent,bottom_percent,dominant_region
0,fold0,ASD_Left_20180521_2_1835_5-5_pred_0,0.340000,0.98,98.680000,下部
1,fold0,ASD_Right_20190917_1942_2-3_pred_1,61.849998,18.15,20.010000,上部
2,fold0,DHS_Left_20171127_994_3-6_pred_0,60.759998,8.87,30.370001,上部
3,fold0,ASD_Left_20190507_2_1610_3-6_pred_1,0.100000,0.74,99.169998,下部
4,fold0,ASD_Right_20190507_2_1645_1-4_pred_1,9.750000,0.84,89.410004,下部
...,...,...,...,...,...,...
2995,fold1,DHS_Right_20200528_1137_5-7_pred_1,98.959999,1.04,0.000000,上部
2996,fold1,ASD_Left_20190507_2_1644_3-7_pred_0,100.000000,0.00,0.000000,上部
2997,fold1,ASD_Right_20171211_1456_6-7_pred_0,90.080002,9.92,0.000000,上部
2998,fold1,DHS_Right_20200528_1124_1-6_pred_0,99.959999,0.04,0.000000,上部




=== Foldごとのサマリー ===

--- Fold 'fold0' ---
[主な注目領域の出現回数]
dominant_region
下部    1505
上部     844
中部      15
Name: count, dtype: int64

[各領域の平均注目率 (%)]
top_percent       28.796760
middle_percent     6.397673
bottom_percent    64.805508
dtype: float64

--- Fold 'fold1' ---
[主な注目領域の出現回数]
dominant_region
上部     2479
N/A     219
下部      166
中部       32
Name: count, dtype: int64

[各領域の平均注目率 (%)]
top_percent       80.171153
middle_percent     4.377624
bottom_percent     7.889068
dtype: float64

--- Fold 'fold2' ---
[主な注目領域の出現回数]
dominant_region
N/A    1081
上部      671
下部      498
中部      223
Name: count, dtype: int64

[各領域の平均注目率 (%)]
top_percent       23.805063
middle_percent    15.195520
bottom_percent    17.287279
dtype: float64

--- Fold 'fold3' ---
[主な注目領域の出現回数]
dominant_region
中部     930
上部     606
N/A    143
下部      40
Name: count, dtype: int64

[各領域の平均注目率 (%)]
top_percent       46.465625
middle_percent    42.347399
bottom_percent     2.868185
dtype: float64

--- Fold 'fold4' ---
[主な注目領域

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2 # 動画ファイルを読み込むために必要

# show_cam_on_image 関数を再定義
# (以前の定義が別のセルにある場合は、そちらのインポートを使うこともできます)
def show_cam_on_image(img: np.ndarray,
                      mask: np.ndarray,
                      use_rgb: bool = False,
                      colormap: int = cv2.COLORMAP_JET) -> np.ndarray:
    """ This function overlays the CAM on the original image.
    Args:
        img (np.ndarray): The original image.
        mask (np.ndarray): The CAM mask.
        use_rgb (bool): Whether to convert the image to RGB.
        colormap (int): The colormap to use for the CAM mask.
    Returns:
        np.ndarray: The image with the CAM overlay.
    """
    heatmap = cv2.applyColorMap(np.uint8(255 * mask), colormap)
    if use_rgb:
        heatmap = cv2.cvtColor(heatmap, cv2.COLOR_BGR2RGB)
    
    heatmap = np.float32(heatmap) / 255
    img = np.float32(img) / 255

    cam = heatmap + img
    cam = cam / np.max(cam)
    return np.uint8(255 * cam)


# --- 【設定】可視化したいファイルとフレームを指定してください ---
# 1. 可視化したい平均CAMの.npyファイルのパス
#    例: 'cam_outputs/fold0/cam_average_test_video_001.mp4_batch_1_vid_0.npy'
NPY_FILE_PATH = '/workspace/data/analysis_logs/fails_points/single_ap/CAM/fold0/cam_average_ASD_Left_20190507_1_1604_2-6.npy' # ★★★変更してください★★★

# 2. そのCAMデータに対応する元の動画ファイルへのパス
#    このパスは、CAM_OUTPUT_DIRではなく、元のデータセットのパスになります。
#    例: '/path/to/your/dataset/test_video_001.mp4'
ORIGINAL_VIDEO_PATH = '/workspace/data/out_dir_ver2_classified_front/fold0/val/ASD/ASD_Left_20190507_1_1604_2-6.mp4' # ★★★変更してください★★★

# 3. 元の動画の何フレーム目を背景画像として使うか (1から8)
#    (平均CAMなのでどのフレームを選んでも意味は同じですが、代表として)
BACKGROUND_FRAME_INDEX = 4 # 1から8の範囲で指定


# --- 1. ファイルの存在確認 ---
if not os.path.exists(NPY_FILE_PATH):
    print(f"エラー: CAMファイルが見つかりません。パスを確認してください: {NPY_FILE_PATH}")
elif not os.path.exists(ORIGINAL_VIDEO_PATH):
    print(f"エラー: 元の動画ファイルが見つかりません。パスを確認してください: {ORIGINAL_VIDEO_PATH}")
else:
    try:
        # --- 2. CAMデータを読み込み ---
        heatmap_data = np.load(NPY_FILE_PATH)
        print(f"CAMファイルを読み込みました。形状: {heatmap_data.shape}")

        # --- 3. 元の動画ファイルを読み込み、指定フレームを抽出 ---
        cap = cv2.VideoCapture(ORIGINAL_VIDEO_PATH)
        if not cap.isOpened():
            raise IOError(f"動画ファイルを開けませんでした: {ORIGINAL_VIDEO_PATH}")

        total_frames_in_video = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        if total_frames_in_video < BACKGROUND_FRAME_INDEX:
            raise ValueError(f"指定されたフレーム {BACKGROUND_FRAME_INDEX} は動画の総フレーム数 {total_frames_in_video} を超えています。")

        # 指定フレームまで読み飛ばし
        cap.set(cv2.CAP_PROP_POS_FRAMES, BACKGROUND_FRAME_INDEX - 1) 
        ret, frame = cap.read() # フレームを読み込み
        cap.release()

        if not ret:
            raise IOError(f"フレーム {BACKGROUND_FRAME_INDEX} を読み込めませんでした。")
        
        # OpenCVはBGRで読み込むのでRGBに変換
        original_image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        print(f"元の画像フレームを読み込みました。形状: {original_image.shape}")

        # --- 4. 元の画像とCAMのサイズを合わせる ---
        # Grad-CAMの出力サイズは通常、モデルの最終層の特徴マップのサイズ（例: 7x7）です。
        # これを元の画像サイズにリサイズする必要があります。
        cam_resized = cv2.resize(heatmap_data, 
                                 (original_image.shape[1], original_image.shape[0]), 
                                 interpolation=cv2.INTER_LINEAR)
        
        # cam_resizedの値が0-1の範囲であることを確認
        cam_resized = (cam_resized - cam_resized.min()) / (cam_resized.max() - cam_resized.min() + 1e-8) # ゼロ除算回避

        # --- 5. show_cam_on_image 関数を使って重ね合わせ ---
        overlayed_image = show_cam_on_image(original_image, cam_resized, use_rgb=True, colormap=cv2.COLORMAP_JET)

        # --- 6. 結果を表示 ---
        plt.figure(figsize=(10, 8))
        plt.imshow(overlayed_image)
        
        # タイトルを設定
        file_name = os.path.basename(NPY_FILE_PATH).replace('.npy', '')
        plt.title(f'Overlayed CAM: {file_name} on Frame {BACKGROUND_FRAME_INDEX}')
        
        # 軸の目盛りを非表示にする
        plt.axis('off')
        
        plt.show()

    except Exception as e:
        print(f"エラーが発生しました。詳細: {e}")

# Grad CAM pytorch
use pytorch_grad_cam api to make visilization.

In [ ]:
#   {
#      'video': <video_tensor>,     # Shape: (C, T, H, W)
#      'audio': <audio_tensor>,     # Shape: (S)
#      'label': <action_label>,     # Integer defining class annotation
#      'video_name': <video_path>,  # Video file path stem
#      'video_index': <video_id>,   # index of video used by sampler
#      'clip_index': <clip_id>      # index of clip sampled within video
#   }
batch = next(iter(test_data))
video = batch['video'].detach() # b, c, t, h, w
label = batch['label'].detach() # b, class num
print("video_path:", batch['video_name'][3])
print("label:", batch['label'])


i=3
input_tensor = video[i].unsqueeze(dim=0).cuda()
inp_label = label[i]

video.shape, label.shape, input_tensor.shape, inp_label

In [ ]:
from pytorch_grad_cam import GradCAM, HiResCAM, FullGrad, GradCAMPlusPlus, AblationCAM, ScoreCAM, LayerCAM
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
from pytorch_grad_cam.utils.image import show_cam_on_image

# guided grad cam method
target_layer = [model.model.blocks[-2].res_blocks[-1]]
# target_layer = [ model.model.blocks[-2]]

cam = LayerCAM(model, target_layer)
y_hat = model.model(video)
y_hat_sigmoid = torch.sigmoid(y_hat)
pred_labels = torch.where(y_hat_sigmoid >= 0.5, torch.tensor(1), torch.tensor(0))
print(pred_labels)
out_label = pred_labels[0]
print(out_label)

targets = [ClassifierOutputTarget(-1)]

grayscale_cam = cam(input_tensor, aug_smooth=True, eigen_smooth=True)

type(grayscale_cam), grayscale_cam.shape

In [ ]:
inp_tensor = input_tensor.squeeze().permute(1,2,3,0)[2].cpu().detach().numpy()
inp_tensor.shape, type(inp_tensor), grayscale_cam.shape

In [ ]:
cam_map = grayscale_cam.mean(axis=1).squeeze()
# cam_map = grayscale_cam[:,:,:,0].squeeze()
cam_map = np.expand_dims(cam_map, 2)

cam_map.shape, type(cam_map), cam_map

In [ ]:
import matplotlib.pyplot as plt 

plt.imshow(cam_map)

In [ ]:
# use captum visual method
figure, axis = viz.visualize_image_attr_multiple(
        cam_map,
        inp_tensor,
        methods=['original_image', 'blended_heat_map'],
        signs=['all', 'positive'],
        show_colorbar=True,
        outlier_perc=1, 
        cmap='jet',
        titles = ['original image', 'GradCAM++, true label %s, pred label %s' % (int(inp_label), int(out_label))]
    )

In [ ]:
input_tensor.shape

In [ ]:
from torchvision.io import write_video

write_video('1s.mp4', video_array=input_tensor.squeeze().permute(1,2,3,0).detach().cpu(), fps=10)

In [ ]:
batch['video_name'][0]